In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://tables.finance.ua/ua/currency/cash/-/ua,0,7oiylpmiow8iy1smadi/eur/1"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

resp = requests.get(url, headers=headers, timeout=30)
resp.raise_for_status()

soup = BeautifulSoup(resp.text, "html.parser")

# Inspect and adjust selectors if the site structure changes.
# Try common table containers used on finance.ua tables:
candidates = []
candidates += soup.select("table")  # fallback to any table
# Prefer tables that look like the main rates list:
preferred = [t for t in candidates if t.get("class") and any("table" in " ".join(t.get("class")) or "rates" in " ".join(t.get("class")) for _ in [0])]

table = preferred[0] if preferred else (candidates[0] if candidates else None)
if table is None:
    raise RuntimeError("Could not locate the rates table on the page.")

# Extract headers
headers_row = table.find("thead")
if headers_row:
    cols = [th.get_text(strip=True) for th in headers_row.select("th")]
else:
    # If no thead, infer from first row
    first_tr = table.find("tr")
    cols = [td.get_text(strip=True) for td in first_tr.find_all(["th", "td"])] if first_tr else []

# Extract body rows
rows_data = []
tbody = table.find("tbody") or table
for tr in tbody.find_all("tr"):
    tds = tr.find_all(["td", "th"])
    if not tds:
        continue
    row = [td.get_text(strip=True) for td in tds]
    # Skip header-duplicate rows
    if cols and row == cols:
        continue
    rows_data.append(row)

# Normalize column count
max_len = max((len(r) for r in rows_data), default=0)
if not cols or len(cols) != max_len:
    cols = cols[:max_len] if cols else [f"col_{i+1}" for i in range(max_len)]
    rows_data = [r[:max_len] + [""] * (max_len - len(r)) for r in rows_data]

df = pd.DataFrame(rows_data, columns=cols)
print(df.head(10))

        Час  Купівля   Продаж                   Назва      Телефон
0  5  12:43  48.4000  49.0000      ІндустріалбанкКиїв  096400 4422
1  5  12:43  48.4000  48.8000     Кліринговий ДімКиїв  044593 1043
2  5  12:43  48.4000  48.9500  Агропросперіс БанкКиїв  044392 9373
3  5  12:43  48.4000  48.9100     Райффайзен БанкКиїв  080050 0500
4  5  12:43  48.5500  48.9100     Кредитвест БанкКиїв  044365 0012
5  5  12:40  48.4000  48.8500        Правекс БанкКиїв  044521 0266
6  5  12:43  48.4000  48.9000                 МІБКиїв  044351 7932
7  5  12:42  48.3000  48.9500            ОщадбанкКиїв  044363 0133
8  5  12:42  48.1000  49.0000          УкрсиббанкКиїв  080050 5800
9  5  12:42  48.4000  49.0000  Банк Кредит ДніпроКиїв  080050 7700


In [5]:
df

,,Час,Купівля,Продаж,Назва,Телефон
0,5,12:43,48.4000,49.0000,ІндустріалбанкКиїв,096400 4422
1,5,12:43,48.4000,48.8000,Кліринговий ДімКиїв,044593 1043
2,5,12:43,48.4000,48.9500,Агропросперіс БанкКиїв,044392 9373
3,5,12:43,48.4000,48.9100,Райффайзен БанкКиїв,080050 0500
4,5,12:43,48.5500,48.9100,Кредитвест БанкКиїв,044365 0012
5,5,12:40,48.4000,48.8500,Правекс БанкКиїв,044521 0266
6,5,12:43,48.4000,48.9000,МІБКиїв,044351 7932
7,5,12:42,48.3000,48.9500,ОщадбанкКиїв,044363 0133
8,5,12:42,48.1000,49.0000,УкрсиббанкКиїв,080050 5800
9,5,12:42,48.4000,49.0000,Банк Кредит ДніпроКиїв,080050 7700
